In [12]:
import os
import shutil

from langchain.schema import BaseMemory
from pydantic import BaseModel
from langchain.vectorstores import DeepLake
from langchain.embeddings import OpenAIEmbeddings
from langchain import OpenAI, LLMMathChain
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool

In [10]:
llm = OpenAI(temperature=0, model_name="gpt-3.5-turbo")
llm_math_chain = LLMMathChain(llm=llm, verbose=True)
tools = [
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    )
]

In [14]:
class EmbeddingMemory(BaseMemory, BaseModel):
    dataset_path = "./vectorstore/"
    os.makedirs(dataset_path, exist_ok=True)
    vector_store = DeepLake(
        dataset_path=dataset_path,
        embedding_function=OpenAIEmbeddings()
    )
    
    @staticmethod
    def init_vector_store(dataset_path: str) -> DeepLake:
        return DeepLake(
            dataset_path=dataset_path,
            embedding_function=OpenAIEmbeddings()
        )
        
    def clear(self):
        shutil.rmtree(self.dataset_path)
        os.makedirs(self.dataset_path)
        self.vector_store = self.init_vector_store(self.dataset_path)
    
    @property
    def memory_variables(self) -> List[str]:
        """Define the variables we are providing to the prompt."""
        return [self.memory_key]
    
    def load_memory_variables(self, inputs: Dict[str, Any]) -> Dict[str, Any]:
        pass
    
    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, str]) -> None:
        pass
    
llm = ChatOpenAI(temperature=0)
memory = EmbeddingMemory()
agent_chain = initialize_agent(tools, llm, agent="chat-conversational-react-description", verbose=True, memory=memory)

ValueError: Could not import chromadb python package. Please install it with `pip install chromadb`.